In [1]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
df = pd.read_csv('/kaggle/input/prediccion-de-sufrir-enfermedades-coronarias/train.csv')
X,y = df.drop(['CHD_OR_MI','ID'],axis=1),df[['CHD_OR_MI']] 

### Feature Engineering

Probamos los siguientes features para ver si estos mejoraban el rendimiento del modelo. Para esto se realizó un cross validation con k fold. Donde k=3. Se descartaron los nuevos features puesto que ninguno aumento el f1_score significativamente.

In [3]:
def evaluate_dataset(x_features,y_features):
    model = LGBMClassifier(verbose=-1)

    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, x_features, y_features, cv=cv, scoring='f1')
    
    print("Scores por fold:", cv_scores)
    print("f1 promedio:", np.mean(cv_scores))
    print("Desviación estándar:", np.std(cv_scores))



In [4]:
evaluate_dataset(X,y.to_numpy().ravel())

Scores por fold: [0.9579628  0.95817918 0.95795822]
f1 promedio: 0.9580333967559514
Desviación estándar: 0.00010309788205868973


In [5]:
X_2 = X.copy()
X_2['AGE_PRESSURE'] = X_2['AGE'] * X_2['BLOOD_PRESSURE'].fillna(0).astype(int)
X_2.loc[X_2['BLOOD_PRESSURE'].isnull(), 'AGE_PRESSURE'] = np.nan
evaluate_dataset(X_2,y.to_numpy().ravel())

Scores por fold: [0.95799895 0.95808416 0.95798829]
f1 promedio: 0.9580238004235112
Desviación estándar: 4.290008858833954e-05


In [6]:
X_3 = X.copy()
X_3['DIABETES_CHOLESTEROL'] = X_3['DIABETES'] * X_3['HIGH_CHOLESTEROL']
evaluate_dataset(X_3,y.to_numpy().ravel())

Scores por fold: [0.95789455 0.95805102 0.957851  ]
f1 promedio: 0.957932188775701
Desviación estándar: 8.588355958022283e-05


In [7]:
X_4 = X.copy()
X_4['ETHNICITY_HEALTH'] = X_4['ETHNICITY'].astype(str) + '_' + X_4['HEALTH'].astype(str)
X_4['ETHNICITY_HEALTH'] = X_4['ETHNICITY_HEALTH'].astype('category')
evaluate_dataset(X_4,y.to_numpy().ravel())

Scores por fold: [0.95794255 0.95810482 0.95797733]
f1 promedio: 0.95800823401037
Desviación estándar: 6.975823782742493e-05


In [8]:
X_5 = X.copy()
X_5['AGE_HIGH'] =  (X_5['AGE']>60).astype(int)
evaluate_dataset(X_5,y.to_numpy().ravel())

Scores por fold: [0.95798563 0.95817918 0.95795822]
f1 promedio: 0.9580410067004262
Desviación estándar: 9.833870015797719e-05


In [9]:
X_6 = X.copy()
X_6['POTATOES_SKIN_CANCER'] = X_6['FRIED_POTATOES'].astype(str) + '_' + X_6['SKIN_CANCER'].astype(str)
X_6['POTATOES_SKIN_CANCER'] = X_6['POTATOES_SKIN_CANCER'].astype('category')
evaluate_dataset(X_6,y.to_numpy().ravel())

Scores por fold: [0.95797695 0.95806333 0.95796302]
f1 promedio: 0.9580011014860462
Desviación estándar: 4.437048017991935e-05


In [10]:
X_7 = X.copy()
X_7['BLOOD_PRESSURE_HIGH'] = (X_7['BLOOD_PRESSURE'] == '2').astype(int)
evaluate_dataset(X_7,y.to_numpy().ravel())

Scores por fold: [0.9579628  0.95817918 0.95795822]
f1 promedio: 0.9580333967559514
Desviación estándar: 0.00010309788205868973


In [11]:
X_8 = X.copy()
scaler = MinMaxScaler()
X_8['BMI_rescaled'] = scaler.fit_transform(X_8[['BMI']]) # Reescalamos BMI antes porque tenia valores muy elevados
X_8['BMIxDIABETES'] = X_8['BMI_rescaled'] * X_8['DIABETES']
evaluate_dataset(X_8,y.to_numpy().ravel())

Scores por fold: [0.95787824 0.9581539  0.95793385]
f1 promedio: 0.9579886634800924
Desviación estándar: 0.00011902384433407037


In [12]:
X_9 = X.copy()
sex_counts = X_9['SEX'].value_counts(normalize=True) 
X_9['SEX_PROPORTION'] = X_9['SEX'].map(sex_counts)
evaluate_dataset(X_9,y.to_numpy().ravel())

Scores por fold: [0.9579628  0.95817918 0.95795822]
f1 promedio: 0.9580333967559514
Desviación estándar: 0.00010309788205868973


In [13]:
X_10 = X.copy()
smoke_counts = X_10['SMOKE'].value_counts(normalize=True) 
X_10['SMOKE_PROPORTION'] = X_10['SMOKE'].map(smoke_counts)
evaluate_dataset(X_10,y.to_numpy().ravel())

Scores por fold: [0.9579628  0.95817918 0.95795822]
f1 promedio: 0.9580333967559514
Desviación estándar: 0.00010309788205868973
